# Resultados

In [1]:
import collections, json, os, time

import numpy as np
import pandas as pd

from dataBase import DataBase

## Metodos

In [2]:
def img_class(path):
    '''
    Retorna a classe da imagem pelo caminho. 
    '''
    
    caminho = path.split(os.path.sep)
    for c in caminho:
        if c in classes:
            return c
    return ''

In [3]:
def analysis_query_result(result, metric, model, img_obj_class):
    '''
    Metodo responsavel por processar cada um dos resultados buscados, retornando
    um json com informacoes relevantes entre a imagem objeto e os resultados da busca
    '''
    k = len(result)
    process = {}
    #pega a path de cada imagem que esta no resultado
    process['imgs_path'] = [x.path for x in result]
    #pega a similaridade de cada uma das imagens
    process['imgs_distance'] = [x.similarity for x in result]
    #pega a classe de cada uma das imagens
    process['imgs_classes'] = [img_class(x.path) for x in result]
    #quantidade de itens recuperados que correspondem a classe da imagem
    process['true_positives'] = collections.Counter(process['imgs_classes'])[img_obj_class]
    #calculo do precision
    process['precision'] = process['true_positives']/k
    #calculo revogacao
    process['recall'] = process['true_positives']/(quantities[img_obj_class]-1)
    #f1 escore
    
    process['f1'] = 2*(process['precision']*process['recall'])/(process['precision']+process['recall'])
    process['metric'] = metric
    process['model'] = model
    
    return process

In [4]:
def short_analysis_query_result(result, metric, model, img_obj_class):
    '''
    Metodo responsavel por processar cada um dos resultados buscados, retornando
    um json com informacoes relevantes entre a imagem objeto e os resultados da busca
    '''
    k = len(result)
    process = {}
    process['k'] = k
    imgs_class = [img_class(x.path) for x in result]
    #quantidade de itens recuperados que correspondem a classe da imagem
    process['true_positive'] = collections.Counter(imgs_class)[img_obj_class]
    #calculo do precision
    process['precision'] = process['true_positive']/k
    #calculo revogacao
    process['recall'] = process['true_positive']/(quantities[img_obj_class]-1)
    #f1 escore
    #process['f1'] = 2*(process['precision']*process['recall'])/(process['precision']+process['recall'])
    process['metric'] = metric
    process['model'] = model
    
    return process

In [5]:
def set_class_quantities(path):
    '''
    Metodo que define a quantidade de itens de cada classe considerando as subpastas
    '''
    quantities= {}
    
    for root, dirs, files in os.walk(path, topdown=False):
        for name in dirs:
            diretorio = os.path.join(root, name)
            dpath, ddirs, dfiles = next(os.walk(diretorio))
            sub = root.replace(path, '').split('/')
            if len(sub) > 1:
                if quantities.get(sub[1]):
                    quantities[sub[1]] += len(dfiles)
                else:
                    quantities[sub[1]] = len(dfiles)
            else:
                if quantities.get(name):
                    quantities[name] += len(dfiles)
                else:
                    quantities[name] = len(dfiles)
            
    return quantities

In [17]:
def evaluate(file, k, df, name_representation, distance_metrics):
    analysis = list()
    
    path = file
    if type(file) != str:
        path = file.path
        
    image_object_class = img_class(path)
    aux = {}
    aux['img'] = path
    aux['class'] = image_object_class
    aux['k'] = k

    for name in name_representation:
        for distance in distance_metrics:
            search_result = db.search(file, model=db.REPRESENTATONS.index(name), k=k+1, distance=distance)
            del search_result[0] #deleta a própria imagem      
            msg = [img_class(x.path) for x in search_result]
            print(collections.Counter(msg)[image_object_class])
            an = short_analysis_query_result(search_result, distance, name, image_object_class)
            aux[name+'_'+distance+'_'+'precision'] = an['precision']
            aux[name+'_'+distance+'_'+'tp'] = an['true_positive']

            analysis.append(an)

    #ediciona os resultados na lista de resultados
    result = {}
    result['image_object'] = path
    result['class_image_object'] = image_object_class
    result['analysis'] = analysis
    results.append(result)

    return pd.DataFrame([aux])

In [7]:
def salvar_resultados(path, df, x):
    with open(path+f'/resultados/json/resultado{x}.json', 'w') as f:
        json.dump(results, f)
        f.close()
    print('json salvo em {}'.format(path+'/resultados/json/'))

    df.to_csv(path+f'/resultados/csv/resultado{x}.csv', index=False, sep=';')
    print('csv salvo em {}.csv'.format(path+'/resultados/csv/'))

In [44]:
def exibir_medias(df):
    #Imprime as medias dos resultados obtidos nas buscas
    
    for distance in distance_metrics:
        print('Metrica de distancia: ', distance)
        for name in name_representation:
            print('Arquitetura ', name)
            a = 0
            for classe in classes: 
                print(classe)
                df_aux = df[df['class'] == classe]
                a += np.mean(df_aux[name+'_'+distance+'_'+'precision'])*100
            print(f"Média para {distance}: {a/4:.1f}%")

In [31]:
def exibir_medias2(df):
    #Imprime as medias dos resultados obtidos nas buscas
    for classe in classes: 
        df_aux = df[df['class'] == classe]
        print("Classe: %s" %(classe))
        print("==============================================")
        for distance in distance_metrics:
            print('Metrica de distancia: ', distance)
            print('----------------------------------------------')
            for name in name_representation:
                print('Arquitetura: ', name)
                print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
                print(f"Precision média: {np.mean(df_aux[name+'_'+distance+'_'+'precision'])*100:.1f}%")
                print(f"Tp média: {np.mean(df_aux[name+'_'+distance+'_'+'tp']):.1f}")
                #print(f"Recall média: {np.mean(df_aux[name+'_'+distance+'_'+'recall']):.2f}")
                print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            print('----------------------------------------------')
        print("==============================================\n\n")

In [9]:
def images_search(path, qtd_imgs):
    """Busca uma determinada quantidade de imagens"""
    images_path = []
    for root, dirs, files in os.walk(path, topdown=False):
        for f in files:
            if qtd_imgs == 0:
                break
            images_path.append(os.path.join(root, f))
            qtd_imgs -= 1
    return images_path
        

In [10]:
def search_test(db, k, name_representation, distance_metrics):
    """Faz a busca entre de k imagens para todas as imagens do banco de dados"""
    df = pd.DataFrame()
    
    for image in db.images:
        df = pd.concat([df, evaluate(file=image, k=k, df=df, 
                        name_representation=name_representation, 
                        distance_metrics=distance_metrics)])
    return df


In [11]:
#diretorios de interesse
path_atual = os.getcwd()
path_models = path_atual.replace('/'+path_atual.split('/')[-1], '')+'/modelos/save_models/'
path_db = path_atual.replace('/'+path_atual.split('/')[-1], '').replace('/'+path_atual.split('/')[-2], '')+'/db'

#nome das CNNs
name_representation = ['vgg16', 'inception', 'xception', 'resNet50']

#metricas de distancia
distance_metrics = ['braycurtis', 'cosine']

#numero de imagens, por conta que tem que retirar a propria imagem que está sendo buscada
quantities = set_class_quantities(path_db)
#classes
classes = list(quantities.keys())

#variaveis auxiliares
results = []
k = 20

In [12]:
#inicializa classe responsável pelos dados
db = DataBase()

In [13]:
db.iniciar_modelos(path_models)

2022-08-12 08:19:42.087374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-12 08:19:42.102390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-12 08:19:42.103625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-12 08:19:42.105660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-12 08:19:42.106282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [18]:
#carrega as imagens de uma pasta

#db.load_representationsDB(path_db)

In [19]:
db.load_representationsJson(path_db+'db33.json', name_representation)

'Carregamento do arquivo finalizado em 5.906247138977051 segundos'

### Teste com imagens não normalizadas

In [20]:
start = time.time() 
print('Iniciando busca para base nao normalizada...')
df = search_test(db, k, name_representation, distance_metrics)
print(f"Busca encerrada em: ", time.time() - start)

Iniciando busca para base nao normalizada...
18
18
4
9
8
7
9
11
20
19
17
16
14
14
16
17
14
19
16
15
14
15
17
16
12
13
17
17
15
16
15
15
16
14
14
13
7
6
15
16
19
19
18
19
18
19
18
19
9
13
13
10
14
15
11
8
16
12
14
14
15
17
11
12
18
18
17
16
16
16
11
15
5
20
17
18
13
13
17
17
17
15
19
16
16
17
18
19
16
15
17
15
16
14
19
18
20
19
13
14
15
15
18
18
13
12
13
10
15
15
13
13
12
14
19
19
18
18
20
20
13
12
17
14
16
17
16
15
17
19
17
17
16
15
9
10
16
17
18
19
17
18
19
19
19
15
16
14
19
20
18
14
14
13
9
9
12
15
12
12
8
12
17
18
15
14
18
17
19
18
12
12
14
15
15
17
18
19
17
15
17
18
19
17
13
14
8
8
13
12
8
9
18
16
17
17
16
16
18
16
6
12
18
16
17
14
17
15
15
18
20
18
17
15
17
19
7
7
1
3
6
5
7
6
11
10
12
13
13
12
11
10
12
20
20
19
15
15
16
16
19
18
14
13
13
13
15
16
18
17
15
15
15
15
18
18
12
13
14
16
15
15
15
16
16
17
16
17
16
16
14
14
17
17
17
16
18
18
17
16
12
8
10
12
15
15
11
10
9
8
11
9
8
8
8
6
16
13
15
13
15
14
13
14
19
19
17
15
15
15
17
17
14
14
18
19
15
16
12
13
19
19
18
19
14
14
16
17
14
15


13
13
6
4
6
7
7
8
8
6
19
18
18
16
17
19
16
17
20
16
15
16
17
18
17
17
13
13
17
17
11
9
8
7
10
8
9
8
16
17
10
9
16
16
15
13
19
18
13
13
16
13
14
15
5
5
10
7
8
11
9
10
6
7
4
4
16
15
6
10
12
13
15
17
14
15
15
11
16
17
19
20
16
15
17
18
12
13
13
13
16
15
13
11
18
16
13
11
16
17
20
20
19
19
18
18
15
12
20
20
17
17
11
14
8
7
12
13
9
7
9
9
15
18
18
17
18
17
15
16
15
11
15
14
10
10
4
5
16
17
16
16
17
17
16
15
6
6
5
7
9
9
7
9
16
17
13
12
15
18
12
15
12
11
7
7
10
10
6
5
19
18
20
17
18
17
16
16
14
15
20
19
15
16
14
17
18
18
17
17
18
18
16
16
1
3
4
10
1
1
6
5
13
19
11
9
16
17
7
7
19
19
20
19
20
19
16
18
17
17
17
17
15
14
14
11
8
7
5
4
3
4
5
3
18
17
14
15
17
17
18
16
18
18
18
17
17
16
15
16
17
17
15
12
12
12
9
10
17
16
5
5
16
15
15
16
12
15
15
18
17
15
17
17
9
9
9
15
8
6
6
8
15
13
15
15
18
18
18
17
18
19
18
18
17
15
18
18
13
11
14
16
14
14
12
12
8
13
6
10
10
10
16
14
12
14
17
17
17
16
13
14
18
16
19
18
17
17
12
13
14
12
16
15
16
17
11
10
14
12
14
13
10
11
11
12
18
18
19
19
17
17
17
16
19
20
18
18
1

5
11
13
18
18
15
17
19
18
19
19
18
18
20
20
13
12
10
11
11
10
8
11
7
15
16
14
14
15
12
11
14
7
15
13
18
17
17
18
11
10
14
13
9
10
17
15
12
13
14
15
11
11
8
11
16
16
12
11
13
14
20
18
11
8
15
14
13
16
17
14
10
8
15
14
14
16
15
17
15
16
8
12
13
12
12
11
8
7
15
14
17
17
18
18
11
8
11
11
9
9
16
12
7
7
11
9
7
8
8
9
11
15
10
11
15
15
7
10
18
17
15
16
19
19
20
19
19
17
14
15
9
10
17
17
5
5
19
20
12
14
15
11
8
8
11
13
5
7
14
12
4
7
4
4
4
5
7
6
10
11
17
18
18
17
15
17
6
7
4
5
5
5
10
11
7
8
9
11
13
13
9
11
13
13
4
4
9
9
13
12
11
12
17
17
19
20
12
15
8
8
15
16
12
13
10
9
17
12
10
12
12
13
7
6
8
9
8
6
8
10
6
8
9
11
10
10
16
16
13
12
6
7
4
3
8
7
8
8
3
1
2
4
3
3
4
5
6
7
6
6
14
13
6
5
5
6
4
3
12
13
4
7
8
9
15
15
17
16
9
12
7
7
5
4
1
1
5
6
3
3
4
3
2
2
3
4
10
10
12
12
12
13
10
10
2
3
6
6
4
4
4
4
17
17
18
19
20
20
17
16
10
9
10
9
12
12
9
10
11
9
14
12
16
19
15
14
8
10
12
13
18
17
18
13
5
6
4
5
4
6
6
8
3
4
3
6
5
5
6
8
7
11
8
10
7
11
11
13
9
11
9
10
12
12
8
9
4
2
8
7
6
9
8
10
6
12
6
10
8
10
7
8
9
10
10
13

8
9
12
14
9
10
11
13
19
19
19
17
17
13
16
16
15
13
19
19
17
17
12
9
16
16
14
15
4
6
4
6
12
9
11
10
3
5
2
3
3
4
6
6
11
11
11
10
17
17
15
13
14
15
6
6
5
7
14
17
11
8
9
5
5
5
11
10
12
13
11
9
13
13
16
17
15
11
14
8
6
5
12
11
17
17
10
3
12
12
16
17
13
12
6
7
15
14
17
17
17
13
8
8
13
14
19
19
8
11
16
13
11
10
16
18
15
13
12
10
17
16
19
18
15
15
13
12
16
18
18
15
15
15
11
11
10
10
9
10
11
12
16
13
11
11
13
14
9
9
7
9
16
17
15
14
17
15
14
14
19
19
17
18
13
12
13
13
6
5
14
15
12
11
14
10
10
9
13
10
14
13
6
6
10
12
15
15
15
10
8
9
9
9
12
13
12
13
17
15
13
12
11
10
16
14
19
20
9
11
15
16
14
6
13
12
12
12
13
14
10
9
13
7
9
9
12
11
7
5
5
5
13
16
18
15
10
10
15
11
8
8
15
16
17
12
13
11
15
16
12
13
15
15
12
13
16
17
15
17
12
8
9
12
8
7
15
14
19
17
12
8
6
6
16
18
14
14
12
14
8
10
17
16
19
17
4
4
14
13
16
17
15
14
10
11
14
16
16
17
14
16
16
13
19
19
15
17
11
12
14
15
17
18
17
18
14
16
12
13
11
11
14
16
15
13
8
9
14
12
17
19
13
13
6
9
10
11
7
6
10
7
11
11
7
8
12
10
15
13
10
7
15
16
9
11
15
14
6
4
12
12

14
14
12
15
14
18
18
18
15
19
18
10
12
13
10
14
13
20
20
18
18
20
20
18
18
17
14
17
16
15
14
18
18
18
19
7
6
12
12
11
11
17
14
15
15
17
18
12
13
17
16
16
15
7
8
14
14
18
16
16
16
10
10
10
10
18
18
18
19
18
18
15
14
20
17
15
16
20
20
17
15
16
14
5
4
20
20
12
12
18
16
12
12
20
20
17
15
9
7
12
11
9
9
15
13
16
15
12
13
15
14
12
12
15
16
15
15
18
18
18
18
11
8
11
14
19
19
14
14
19
20
9
10
13
12
13
9
19
18
10
14
13
13
19
17
9
13
9
10
15
15
10
11
20
18
18
16
19
19
19
16
18
15
11
8
18
17
13
16
20
20
18
16
20
20
19
18
12
10
7
6
8
6
9
6
19
18
11
14
18
17
17
18
17
17
17
16
20
20
17
18
18
19
11
11
17
17
18
17
15
13
14
11
13
12
15
13
14
14
10
9
11
12
11
11
19
19
16
15
20
20
16
17
16
18
13
11
18
18
14
13
20
17
14
16
20
18
17
15
13
12
14
14
12
12
12
9
18
17
17
17
20
20
17
17
16
13
12
11
8
5
10
8
14
16
11
12
18
17
13
13
18
17
12
14
18
18
13
12
18
18
12
11
19
19
13
14
13
15
5
10
20
20
14
12
15
12
9
8
18
18
10
10
18
18
16
17
18
17
16
17
19
19
15
14
20
20
15
16
20
20
12
12
16
16
18
18
16
17
12
13
19
20
1

18
5
4
5
6
13
14
12
13
15
17
10
7
11
10
18
14
14
13
16
16
14
14
18
14
15
12
14
15
9
7
12
12
9
11
7
9
8
9
13
12
16
14
14
14
10
10
18
16
14
12
15
15
14
15
18
14
10
13
8
10
9
10
11
8
9
12
13
15
15
13
16
16
12
8
16
15
12
12
11
16
12
11
3
6
8
7
5
9
12
10
13
13
8
8
15
13
8
6
4
4
9
8
16
15
12
11
15
15
18
14
13
14
16
15
10
13
16
16
19
16
13
17
13
13
15
11
13
14
15
15
14
12
14
14
12
5
13
12
8
9
7
7
17
17
17
16
12
12
12
10
17
13
14
12
10
8
14
11
15
14
16
17
11
13
12
13
13
12
13
13
10
11
12
11
12
8
16
14
12
15
7
7
14
13
8
5
10
11
10
10
15
13
4
5
8
8
15
12
18
20
13
9
7
8
9
7
8
10
8
6
10
9
7
6
17
14
9
10
10
12
12
12
15
12
12
10
8
11
17
12
17
10
11
12
11
11
12
14
15
13
3
1
8
6
13
12
14
12
11
9
4
4
4
4
17
14
3
4
4
4
4
6
11
11
14
12
14
15
10
9
15
16
12
15
18
19
11
7
17
14
17
15
9
10
11
9
16
18
16
15
19
18
13
11
13
12
6
11
10
9
5
7
15
12
10
13
15
15
10
10
11
5
7
5
7
6
5
5
15
14
15
14
12
15
7
11
17
15
16
16
14
14
9
9
15
15
2
1
1
1
1
1
16
15
16
16
12
15
15
13
17
16
13
11
8
7
10
11
9
7
11
10
9
9
8
9
4
5
1

17
17
7
7
11
10
8
9
5
5
12
9
16
14
15
10
10
11
12
11
12
10
14
11
9
12
12
8
9
10
6
5
7
8
7
6
14
9
9
8
10
9
10
8
3
6
3
1
4
5
2
2
14
16
8
7
14
16
9
9
8
7
10
9
10
9
11
9
6
8
6
6
7
5
3
5
5
5
5
7
6
7
3
5
3
5
6
2
3
3
2
4
19
19
18
17
15
14
10
10
8
7
6
5
6
7
8
8
17
16
14
12
15
16
17
16
7
8
17
16
13
15
10
11
16
11
8
7
9
10
4
6
8
3
3
6
10
7
2
2
9
10
10
10
12
9
4
3
14
8
15
14
14
14
7
5
17
16
14
11
10
10
12
11
6
2
11
13
2
3
3
3
17
16
15
15
13
15
15
16
16
15
18
18
14
15
7
9
12
12
8
11
6
5
5
4
14
11
15
14
11
10
6
5
15
11
12
12
12
9
4
1
14
12
11
12
8
8
6
6
12
13
6
4
6
7
3
2
16
12
12
9
9
10
5
3
9
8
6
4
11
11
11
11
9
7
14
12
7
7
6
3
16
14
16
17
14
15
11
10
13
10
3
4
7
7
8
6
13
9
11
12
10
11
11
9
15
9
12
12
17
16
12
11
9
9
9
6
7
7
8
7
16
11
13
12
14
13
6
5
3
3
4
4
2
3
4
4
5
6
7
6
7
8
4
5
12
5
12
13
10
10
6
6
9
4
10
7
7
7
12
12
8
5
3
2
6
6
7
5
7
4
11
9
10
10
5
2
5
12
17
17
11
11
10
11
12
8
13
15
1
2
16
14
12
12
12
10
14
15
10
8
17
15
15
17
10
13
17
16
3
4
3
2
12
11
6
6
13
14
12
13
12
10
8
7
3
5
3
2
1
2
1


In [47]:
classes.removeand df[df['class'] != '.ipynb_checkpoints'

['hypercellularity', 'normal', 'membranous', 'sclerosis', '.ipynb_checkpoints']

In [45]:
exibir_medias(df)

Metrica de distancia:  braycurtis
Arquitetura  vgg16
hypercellularity
normal
membranous
sclerosis
.ipynb_checkpoints
Média para braycurtis: nan%
Arquitetura  inception
hypercellularity
normal
membranous
sclerosis
.ipynb_checkpoints
Média para braycurtis: nan%
Arquitetura  xception
hypercellularity
normal
membranous
sclerosis
.ipynb_checkpoints
Média para braycurtis: nan%
Arquitetura  resNet50
hypercellularity
normal
membranous
sclerosis
.ipynb_checkpoints
Média para braycurtis: nan%
Metrica de distancia:  cosine
Arquitetura  vgg16
hypercellularity
normal
membranous
sclerosis
.ipynb_checkpoints
Média para cosine: nan%
Arquitetura  inception
hypercellularity
normal
membranous
sclerosis
.ipynb_checkpoints
Média para cosine: nan%
Arquitetura  xception
hypercellularity
normal
membranous
sclerosis
.ipynb_checkpoints
Média para cosine: nan%
Arquitetura  resNet50
hypercellularity
normal
membranous
sclerosis
.ipynb_checkpoints
Média para cosine: nan%


In [32]:
exibir_medias2(df)

Classe: hypercellularity
Metrica de distancia:  braycurtis
----------------------------------------------
Arquitetura:  vgg16
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Precision média: 73.4%
Tp média: 14.7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Arquitetura:  inception
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Precision média: 71.7%
Tp média: 14.3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Arquitetura:  xception
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Precision média: 70.4%
Tp média: 14.1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Arquitetura:  resNet50
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Precision média: 68.7%
Tp média: 13.7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
----------------------------------------------
Metrica de distancia:  cosine
----------------------------------------------
Arquitetura:  vgg16
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Precision média: 73.5%
Tp média: 14.7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Arquitetura:  inception
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
salvar_resultados(path_atual, df, 'kimia')

In [ ]:
print('Processo Finalizado!')
